# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import nltk
nltk.download(['punkt', 'wordnet'])
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
# import statements
import pandas as pd
from sqlalchemy import create_engine
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import make_scorer, accuracy_score, f1_score, fbeta_score, classification_report
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import GridSearchCV
from sklearn import tree
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import precision_recall_fscore_support

[nltk_data] Downloading package punkt to /home/lucy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/lucy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql('disaster_data',engine)
X = df['message']
y = df.iloc[:,4:]
categories = y.columns

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    cleaned_tokens = []
    for i in tokens:
        clean_token = lemmatizer.lemmatize(i).lower().strip()
        cleaned_tokens.append(clean_token)

    return cleaned_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 23)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
y_pred = pipeline.predict(X_test)

In [7]:
def performance_report(model, X_test, y_test, category_names):
    # predict test df
    y_pred = model.predict(X_test)
    total_accuracy = 0
    total_f1 = 0
    # print report 
    for i, category in enumerate(category_names):    
        class_report =  classification_report(y_test[y_test.columns[i]], y_pred[:,i])
        total_accuracy += accuracy_score(y_test[y_test.columns[i]], y_pred[:,i])
        total_f1 += precision_recall_fscore_support(y_test[y_test.columns[i]], y_pred[:,i], average = 'weighted')[2]
        print(category, 'accuracy: {:.5f}'.format(accuracy_score(y_test[y_test.columns[i]], y_pred[:,i])))
        print(class_report)
    print('total accuracy {:.5f}'.format(total_accuracy/len(category_names)))
    print('total f1 {:.5f}'.format(total_f1/len(category_names)))

In [8]:
category_names = list(df.columns[4:])
performance_report(pipeline, X_test, y_test, category_names)

related accuracy: 0.79829
              precision    recall  f1-score   support

           0       0.73      0.27      0.39      1537
           1       0.80      0.97      0.88      4968
           2       0.67      0.08      0.15        49

    accuracy                           0.80      6554
   macro avg       0.73      0.44      0.47      6554
weighted avg       0.79      0.80      0.76      6554

request accuracy: 0.88557
              precision    recall  f1-score   support

           0       0.88      0.99      0.93      5390
           1       0.91      0.40      0.55      1164

    accuracy                           0.89      6554
   macro avg       0.90      0.69      0.74      6554
weighted avg       0.89      0.89      0.87      6554

offer accuracy: 0.99619
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6529
           1       0.00      0.00      0.00        25

    accuracy                           1.00      655

### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x7fe184bf2b90>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          ccp_alpha=0.0,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_depth=No

In [10]:
parameters = {
            'tfidf__use_idf': (True, False), 'tfidf__smooth_idf': (True, False),
            'tfidf__sublinear_tf': (True, False)}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=10)
cv.fit(X,y)
print(cv.best_score_)
print(cv.best_params_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] tfidf__smooth_idf=True, tfidf__sublinear_tf=True, tfidf__use_idf=True 
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV]  tfidf__smooth_idf=True, tfidf__sublinear_tf=True, tfidf__use_idf=True, score=0.250, total= 5.3min
[CV] tfidf__smooth_idf=True, tfidf__sublinear_tf=True, tfidf__use_idf=True 
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  5.3min remaining:    0.0s
[CV]  tfidf__smooth_idf=True, tfidf__sublinear_tf=True, tfidf__use_idf=True, score=0.319, total= 5.6min
[CV] tfidf__smooth_idf=True, tfidf__sublinear_tf=True, tfidf__use_idf=True 
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 10.9min remaining:    0.0s
[CV]  tfidf__smooth_idf=True, tfidf__sublinear_tf=True, tfidf__use_idf=True, score=0.175, total= 5.0min
[CV] tfidf__smooth_idf=True, tfidf__sublinear_tf=True, tfidf__use_idf=True 
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 15.9min remaining:    0.0s
[CV

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
performance_report(cv, X_test, y_test, category_names)

related accuracy: 0.99893
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1537
           1       1.00      1.00      1.00      4968
           2       1.00      0.96      0.98        49

    accuracy                           1.00      6554
   macro avg       1.00      0.99      0.99      6554
weighted avg       1.00      1.00      1.00      6554

request accuracy: 0.99969
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5390
           1       1.00      1.00      1.00      1164

    accuracy                           1.00      6554
   macro avg       1.00      1.00      1.00      6554
weighted avg       1.00      1.00      1.00      6554

offer accuracy: 1.00000
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6529
           1       1.00      1.00      1.00        25

    accuracy                           1.00      655

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [15]:
pipeline1 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier()))])

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 23)
pipeline1.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at 0x7fe184bf2b90>,
                                 vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R'

In [18]:
y_pred = pipeline1.predict(X_test)
performance_report(pipeline1, X_test, y_test, category_names)

related accuracy: 0.76869
              precision    recall  f1-score   support

           0       0.57      0.18      0.28      1537
           1       0.79      0.96      0.86      4968
           2       0.42      0.20      0.27        49

    accuracy                           0.77      6554
   macro avg       0.59      0.45      0.47      6554
weighted avg       0.73      0.77      0.72      6554

request accuracy: 0.88831
              precision    recall  f1-score   support

           0       0.91      0.96      0.93      5390
           1       0.76      0.55      0.63      1164

    accuracy                           0.89      6554
   macro avg       0.83      0.75      0.78      6554
weighted avg       0.88      0.89      0.88      6554

offer accuracy: 0.99512
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6529
           1       0.00      0.00      0.00        25

    accuracy                           1.00      655

### 9. Export your model as a pickle file

In [13]:
import joblib

In [14]:
joblib.dump(cv.best_estimator_, 'disaster_model1.pkl')

['disaster_model1.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [89]:
#Trying more parameters
parameters = {
            'tfidf__use_idf': (True, False), 'tfidf__smooth_idf': (True, False),
            'tfidf__sublinear_tf': (True, False), 'tfidf__use_idf': (True, False), 'clf__estimator__max_depth': [10, 50, None],
              'clf__estimator__min_samples_leaf':[2, 5, 10]}

cv = GridSearchCV(pipeline, param_grid=parameters)
cv.fit(X,y)
print(cv.best_score_)
print(cv.best_params_)

0.19545238698999778
{'clf__estimator__max_depth': None, 'clf__estimator__min_samples_leaf': 2, 'tfidf__smooth_idf': True, 'tfidf__sublinear_tf': False, 'tfidf__use_idf': True}
